In [21]:
!pip install nerda -q
!pip install seqeval -q
!pip install nerda -q
!pip install flair -q


     |████████████████████████████████| 319 kB 15.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 48 kB 6.0 MB/s 
     |████████████████████████████████| 64 kB 3.4 MB/s 
     |████████████████████████████████| 788 kB 65.6 MB/s 
     |████████████████████████████████| 19.7 MB 88 kB/s 
     |████████████████████████████████| 1.2 MB 75.2 MB/s 
     |████████████████████████████████| 981 kB 68.7 MB/s 
     |████████████████████████████████| 62 kB 1.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
from NERDA.datasets import get_conll_data, download_conll_data 
from google.colab import files
import pandas as pd

import unicodedata

import numpy as np
import seqeval.metrics
import spacy
import torch
from tqdm import tqdm, trange
from transformers import LukeTokenizer, LukeForEntitySpanClassification
from flair.data import Sentence
from flair.models import SequenceTagger
import timeit
from sklearn.model_selection import train_test_split 

uploaded = files.upload()


download_conll_data()
training = get_conll_data('train')
validation = get_conll_data('valid')
testing = get_conll_data('test')

Saving processed_df.csv to processed_df.csv
Saving retrain_processed.csv to retrain_processed.csv
Reading https://data.deepai.org/conll2003.zip


In [3]:
# Download the testb set of the CoNLL-2003 dataset
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb

--2021-10-22 15:31:36--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748096 (731K) [text/plain]
Saving to: ‘eng.testb’

eng.testb           100%[===================>] 730.56K  --.-KB/s    in 0.02s   

2021-10-22 15:31:36 (44.8 MB/s) - ‘eng.testb’ saved [748096/748096]



In [4]:
tag_scheme = [
'B-PER',
'I-PER',
'B-ORG',
'I-ORG',
'B-LOC',
'I-LOC',
'B-MISC',
'I-MISC'
]

In [5]:
def generate_labels(input_text):
  input_text = str(input_text)
  if input_text.count(" ") > 0:
    if "went to the store" in input_text:
      if input_text.count(" ") > 4:
        return ["B-PER", "I-PER", "O", "O", "O", "O"]
      return ["B-PER", "O", "O", "O", "O"]
    return ["B-PER", "I-PER"]
  else: 
    return ["B-PER"]

In [7]:
transformer = 'studio-ousia/luke-large-finetuned-conll-2003'

# hyperparameters for network
dropout = 0.1

# hyperparameters for training
# training_hyperparameters = {
# 'epochs' : 5,
# 'warmup_steps' : 2500,                                                   
# 'train_batch_size': 2048,                                         
# 'learning_rate': 1e-5
# }

training_hyperparameters = {
'epochs' : 2,
'warmup_steps' : 500,                                                   
'train_batch_size': 13,                                         
'learning_rate': 1e-5
}

In [8]:
def get_sentence_from_name(input_name):
  input_name = str(input_name)
  return input_name.split(" ")

In [11]:
retrain_subset = pd.read_csv("retrain_processed.csv", index_col=0)
retrain_subset["tags_list"] = retrain_subset["Name"].apply(lambda x: generate_labels(x))
retrain_subset["sentences"] = retrain_subset["Name"].apply(lambda x: get_sentence_from_name(x))

rt_train, rt_valid = train_test_split(retrain_subset, test_size=0.15, stratify=retrain_subset['Race']) 

In [14]:
retrain_subset = pd.read_csv("retrain_processed.csv", index_col=0)
retrain_subset["tags_list"] = retrain_subset["Name"].apply(lambda x: generate_labels(x))
retrain_subset["sentences"] = retrain_subset["Name"].apply(lambda x: get_sentence_from_name(x))

In [15]:
retrain_dict = {"sentences": list(rt_train["sentences"]), "tags": list(rt_train["tags_list"])}
valid_dict = {"sentences": list(rt_valid["sentences"]), "tags": list(rt_valid["tags_list"])}

In [16]:
total_sentences = list(retrain_dict["sentences"]) + list(training["sentences"])
total_tags = list(retrain_dict["tags"]) + list(training["tags"])

valid_sentences = list(valid_dict["sentences"]) + list(validation["sentences"])
valid_tags = list(valid_dict["tags"]) + list(validation["tags"])

total_retrain_dict = {"sentences": total_sentences, "tags": total_tags}
total_valid_dict = {"sentences": valid_sentences, "tags": valid_tags}


# LUKE Model Finetuning

In [24]:
from NERDA.models import NERDA
model = NERDA(
dataset_training = total_retrain_dict,
dataset_validation = total_valid_dict,
tag_scheme = tag_scheme, 
tag_outside = 'O',
transformer = transformer,
dropout = dropout,
hyperparameters = training_hyperparameters
)

In [ ]:
model.train()

# Custom LUKE Model Testing on CoNLL 2003 Test Set


In [ ]:
# test = get_conll_data('test')
# model.evaluate_performance(test)

# Custom LUKE Model Testing on Curated Name Dataset


In [26]:
processed_test_df = pd.read_csv("processed_df.csv", index_col=0)
processed_test_df["tags_list"] = processed_test_df["Name"].apply(lambda x: generate_labels(x))
processed_test_df["sentences"] = processed_test_df["Name"].apply(lambda x: get_sentence_from_name(x))
processed_test_dict = {"sentences": list(processed_test_df["sentences"]), "tags": list(processed_test_df["tags_list"])}

In [ ]:
# model.evaluate_performance(processed_test_dict)

In [ ]:
# curated_test_labels = processed_test_dict["tags"]
# curated_pred_labels = [generate_entities(processed_test_dict["sentences"][i]) for i in range(len(processed_test_dict["sentences"]))]
# # curated_pred_labels = [i[0] for i in curated_pred_labels]

In [18]:
# print(seqeval.metrics.classification_report(curated_test_labels, curated_pred_labels, digits=4)) 

In [25]:
# load tagger
tagger = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2021-10-22 15:36:30,982 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

In [54]:
def generate_entities(input_string):
  return model.predict([input_string])[0]

In [55]:
processed_test_df["Race"].unique()

array(['White', 'Black', 'API', 'Hispanic'], dtype=object)

In [62]:
processed_white_df = processed_test_df.loc[processed_test_df["Race"]=="White"].reset_index(drop=True)
processed_black_df = processed_test_df.loc[processed_test_df["Race"]=="Black"].reset_index(drop=True)
processed_api_df = processed_test_df.loc[processed_test_df["Race"]=="API"].reset_index(drop=True)
processed_hispanic_df = processed_test_df.loc[processed_test_df["Race"]=="Hispanic"].reset_index(drop=True)

processed_test_dict_w = {"sentences": list(processed_white_df["sentences"]), "tags": list(processed_white_df["tags_list"])}
processed_test_dict_b = {"sentences": list(processed_black_df["sentences"]), "tags": list(processed_black_df["tags_list"])}
processed_test_dict_a = {"sentences": list(processed_api_df["sentences"]), "tags": list(processed_api_df["tags_list"])}
processed_test_dict_h = {"sentences": list(processed_hispanic_df["sentences"]), "tags": list(processed_hispanic_df["tags_list"])}

# Flair Baseline Results

In [64]:
start = timeit.default_timer()
curated_test_labels_w = processed_test_dict_w["tags"]
curated_pred_labels_w = [generate_entities_flair_baseline(processed_test_dict_w["sentences"][i]) for i in range(len(processed_test_dict_w["sentences"]))]
print(seqeval.metrics.classification_report(curated_test_labels_w, curated_pred_labels_w, digits=4)) 
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000         0
         ORG     0.0000    0.0000    0.0000         0
         PER     0.9941    0.9709    0.9823      8480

   micro avg     0.9907    0.9709    0.9807      8480
   macro avg     0.3314    0.3236    0.3274      8480
weighted avg     0.9941    0.9709    0.9823      8480

Flair Runtime: 128.63406354699987 seconds


In [65]:
start = timeit.default_timer()
curated_test_labels_b = processed_test_dict_b["tags"]
curated_pred_labels_b = [generate_entities_flair_baseline(processed_test_dict_b["sentences"][i]) for i in range(len(processed_test_dict_b["sentences"]))]
print(seqeval.metrics.classification_report(curated_test_labels_b, curated_pred_labels_b, digits=4)) 
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000         0
        MISC     0.0000    0.0000    0.0000         0
         ORG     0.0000    0.0000    0.0000         0
         PER     0.9770    0.7429    0.8440      8476

   micro avg     0.8718    0.7429    0.8022      8476
   macro avg     0.2443    0.1857    0.2110      8476
weighted avg     0.9770    0.7429    0.8440      8476

Flair Runtime: 128.15284399300003 seconds


In [66]:
start = timeit.default_timer()
curated_test_labels_a = processed_test_dict_a["tags"]
curated_pred_labels_a = [generate_entities_flair_baseline(processed_test_dict_a["sentences"][i]) for i in range(len(processed_test_dict_a["sentences"]))]
print(seqeval.metrics.classification_report(curated_test_labels_a, curated_pred_labels_a, digits=4)) 
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000         0
         ORG     0.0000    0.0000    0.0000         0
         PER     0.9567    0.7300    0.8281      8468

   micro avg     0.9119    0.7300    0.8109      8468
   macro avg     0.3189    0.2433    0.2760      8468
weighted avg     0.9567    0.7300    0.8281      8468

Flair Runtime: 127.62451069199983 seconds


In [67]:
start = timeit.default_timer()
curated_test_labels_h = processed_test_dict_h["tags"]
curated_pred_labels_h = [generate_entities_flair_baseline(processed_test_dict_h["sentences"][i]) for i in range(len(processed_test_dict_h["sentences"]))]
print(seqeval.metrics.classification_report(curated_test_labels_h, curated_pred_labels_h, digits=4)) 
stop = timeit.default_timer()
print('Flair Runtime: {} seconds'.format(stop - start))

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000         0
        MISC     0.0000    0.0000    0.0000         0
         ORG     0.0000    0.0000    0.0000         0
         PER     0.9955    0.9580    0.9764      8480

   micro avg     0.9815    0.9580    0.9696      8480
   macro avg     0.2489    0.2395    0.2441      8480
weighted avg     0.9955    0.9580    0.9764      8480

Flair Runtime: 128.02982846999998 seconds


# Custom LUKE Model Testing on CoNLL 2003 Test Set


In [69]:
# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

def load_documents(dataset_file):
    documents = []
    words = []
    labels = []
    sentence_boundaries = []
    with open(dataset_file) as f:
        for line in f:
            line = line.rstrip()
            if line.startswith("-DOCSTART"):
                if words:
                    documents.append(dict(
                        words=words,
                        labels=labels,
                        sentence_boundaries=sentence_boundaries
                    ))
                    words = []
                    labels = []
                    sentence_boundaries = []
                continue

            if not line:
                if not sentence_boundaries or len(words) != sentence_boundaries[-1]:
                    sentence_boundaries.append(len(words))
            else:
                items = line.split(" ")
                words.append(items[0])
                labels.append(items[-1])

    if words:
        documents.append(dict(
            words=words,
            labels=labels,
            sentence_boundaries=sentence_boundaries
        ))
        
    return documents


def load_examples(documents):
    examples = []
    max_token_length = 510
    max_mention_length = 30

    for document in tqdm(documents):
        words = document["words"]
        subword_lengths = [len(tokenizer.tokenize(w)) for w in words]
        total_subword_length = sum(subword_lengths)
        sentence_boundaries = document["sentence_boundaries"]

        for i in range(len(sentence_boundaries) - 1):
            sentence_start, sentence_end = sentence_boundaries[i:i+2]
            if total_subword_length <= max_token_length:
                # if the total sequence length of the document is shorter than the
                # maximum token length, we simply use all words to build the sequence
                context_start = 0
                context_end = len(words)
            else:
                # if the total sequence length is longer than the maximum length, we add
                # the surrounding words of the target sentence　to the sequence until it
                # reaches the maximum length
                context_start = sentence_start
                context_end = sentence_end
                cur_length = sum(subword_lengths[context_start:context_end])
                while True:
                    if context_start > 0:
                        if cur_length + subword_lengths[context_start - 1] <= max_token_length:
                            cur_length += subword_lengths[context_start - 1]
                            context_start -= 1
                        else:
                            break
                    if context_end < len(words):
                        if cur_length + subword_lengths[context_end] <= max_token_length:
                            cur_length += subword_lengths[context_end]
                            context_end += 1
                        else:
                            break

            text = ""
            for word in words[context_start:sentence_start]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "

            sentence_words = words[sentence_start:sentence_end]
            sentence_subword_lengths = subword_lengths[sentence_start:sentence_end]

            word_start_char_positions = []
            word_end_char_positions = []
            for word in sentence_words:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                word_start_char_positions.append(len(text))
                text += word
                word_end_char_positions.append(len(text))
                text += " "

            for word in words[sentence_end:context_end]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "
            text = text.rstrip()

            entity_spans = []
            original_word_spans = []
            for word_start in range(len(sentence_words)):
                for word_end in range(word_start, len(sentence_words)):
                    if sum(sentence_subword_lengths[word_start:word_end]) <= max_mention_length:
                        entity_spans.append(
                            (word_start_char_positions[word_start], word_end_char_positions[word_end])
                        )
                        original_word_spans.append(
                            (word_start, word_end + 1)
                        )

            examples.append(dict(
                text=text,
                words=sentence_words,
                entity_spans=entity_spans,
                original_word_spans=original_word_spans,
            ))

    return examples


def is_punctuation(char):
    cp = ord(char)
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/877 [00:00<?, ?B/s]

In [70]:
test_documents = load_documents("eng.testb")
test_examples = load_examples(test_documents)

100%|██████████| 231/231 [00:03<00:00, 74.09it/s]


In [ ]:
def get_named_entities_custom_luke(input_row):
  words = input_row["words"]
  sentence_boundaries = input_row["sentence_boundaries"]
  start = 0
  total_labels = []
  for i in sentence_boundaries: 
    if i != 0: 
      current_string = words[start:i]
      if len(current_string) >= 120:
        midpoint = len(current_string) // 2
        first_half = current_string[:midpoint]
        second_half = current_string[midpoint:]
        prediction = model.predict([first_half])[0] + model.predict([second_half])[0]
      else: 
        prediction = model.predict([current_string])[0]
      total_labels.extend(prediction)
      start = i
  return total_labels

In [264]:
def get_named_entities_flair_baseline(input_row):
  words = input_row["words"]
  sentence_boundaries = input_row["sentence_boundaries"]
  start = 0
  total_labels = []

  for i in sentence_boundaries: 
    if i != 0: 
      current_string = words[start:i]
      # print(current_string)
      # print(len(current_string))
      if len(current_string) >= 120:
        midpoint = len(current_string) // 2
        first_half = current_string[:midpoint]
        second_half = current_string[midpoint:]
        prediction = generate_entities_flair_baseline(first_half) + generate_entities_flair_baseline(second_half)
      else: 
        prediction = generate_entities_flair_baseline(current_string)
      total_labels.extend(prediction)
      start = i
  return total_labels

In [225]:
# model.save_network(model_path='model.bin')
# files.download('model.bin') 

In [265]:
def generate_entities_flair_baseline(input_string):
    input_string = " ".join(input_string)
    sentence = Sentence(input_string)
    # print(sentence)
    # predict NER tags
    tagger.predict(sentence)
    sentence_length = len(sentence)
    values = ["O"] * len(input_string.split(" "))
    total_string = ""
    tagged_string = sentence.to_tagged_string()
    true_index = 0
    # print(tagged_string)
    count_entities = 0
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

    # print(tagged_string.split(">"))
    # print(sentence.to_dict(tag_type='ner'))
    tagged_dict = sentence.to_dict(tag_type='ner')
    named_entities = tagged_dict["entities"]
    total_entities = []
    total_text = []

    for i in named_entities:
      text = named_entities[0]["text"]
      space_count = text.count(" ")

      entities = []
      current_entity = str(i["labels"][0])[:5]
      current_text = str(i["text"])

      if "ORG" in current_entity:
        current_entity = "ORG"
      if "MISC" in current_entity:
        current_entity = "MISC"
      if "PER" in current_entity:
        current_entity = "PER"
      if "LOC" in current_entity:
        current_entity = "LOC"
      total_text.append(current_text)
      entities.append("B-"+current_entity)
      if space_count >=1: 
        for i in range(space_count):
          entities.append("I-"+current_entity)
      total_entities.append(entities)

    # print(total_entities)
    copy_string = input_string

    for i, te in enumerate(total_text):
        copy_string = copy_string.replace(te, str(total_entities[i]).replace(" ", ""), 1)
            
    entity_list = []
    # print(copy_string)

    for i in copy_string.split(" "):
        prefix = (i[0:4])
        if prefix == "['B-":
          entry = [n.strip() for n in ast.literal_eval(i)]
          entity_list.extend(entry)
        else:
            entity_list.append("O")
    return entity_list

In [253]:
# get_named_entities_flair_baseline([["hello", "I'm", "David"]])

TypeError: ignored

In [266]:
test_labels = [test_documents[i]["labels"] for i in range(len(test_documents[:50]))]
pred_labels = [get_named_entities_flair_baseline(test_documents[i]) for i in range(len(test_documents[:50]))]
# pred_labels = [get_named_entities_custom_luke(test_documents[i]) for i in range(len(test_documents))]

23
13
15
27
18
47
6
8
8
34
16
9
15
7
17
40
5
4
5
17
7
28
7
50
13
18
55
68
6
9
26
29
14
9
26
7
16
5
12
59
6
29
6
15
9
34
16
8
6
12


In [228]:
len(test_documents[54]["labels"])

693

In [288]:
test_documents[13]["words"]

['SOCCER',
 '-',
 'BRAZILIAN',
 'CHAMPIONSHIP',
 'RESULTS',
 '.',
 'RIO',
 'DE',
 'JANEIRO',
 '1996-12-05',
 'Results',
 'of',
 'Brazilian',
 'soccer',
 'championship',
 'semifinal',
 ',',
 'first',
 'leg',
 'matches',
 'on',
 'Thursday',
 '.',
 'Goias',
 '1',
 'Gremio',
 '3',
 'Portuguesa',
 '1',
 'Atletico',
 'Mineiro',
 '0']

In [290]:
test_labels[13]

['O',
 'O',
 'I-MISC',
 'O',
 'O',
 'O',
 'I-LOC',
 'I-LOC',
 'I-LOC',
 'O',
 'O',
 'O',
 'I-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-ORG',
 'O',
 'I-ORG',
 'O',
 'I-ORG',
 'O',
 'I-ORG',
 'I-ORG',
 'O']

In [289]:
pred_labels[13]

['O',
 'O',
 'B-MISC',
 'O',
 'O',
 'O',
 'B-LOC',
 'I-LOC',
 'I-LOC',
 'O',
 'O',
 'O',
 'B-MISC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O']

In [287]:
[i for i in range(len(pred_labels)) if len(pred_labels[i]) == 31]

[13]

In [267]:
for i, val in enumerate(test_labels): 
  if len(val) != len(pred_labels[i]):
    print(pred_labels[i])
    print(val)
    print(len(test_documents[i]["words"]))
    print(len(val))
    print(len(pred_labels[i]))

    print("-----")

['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'B-LOC', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '

In [ ]:
print(seqeval.metrics.classification_report(test_labels, pred_labels, digits=4)) 

# Baseline LUKE Model Testing on Curated Name Dataset


# Baseline LUKE Model Testing on CoNLL 2003 Dataset
